In [1]:
import pickle
import numpy as np
import pandas as pd
from itertools import combinations
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import os



class SingleFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image

pd.set_option("display.max_rows",1000000000)

In [2]:
# Load efficient net model
with open('prob_efficient.pickle', 'rb') as f:
    eff_prob = pickle.load(f)

name_eff = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5']

prob_eff = []
for i in range(eff_prob.shape[0]):
    prob_eff.append(eff_prob[i])

In [3]:
# Load res50 and yolov8
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dataset = SingleFolderDataset('test', transform=valid_transform)

test_num = list()
for file_name in test_dataset.image_list:
    test_num.append(int(file_name.split('.')[0]))
test_indices = sorted(range(len(test_num)), key=lambda k: test_num[k])


with open("yolov8-prob-vector", 'rb') as file:
    yoloV = pickle.load(file)

yoloV = yoloV.cpu().numpy()[test_indices,:]

In [4]:
def essemble_comb(comb_indx,
                  name_list,
                  prob_list):
    cm_n = len(comb_indx)
    index_name = name_list[comb_indx[0]]
    for k in range(1,cm_n):
        index_name = index_name + name_list[comb_indx[k]]
    comb_prob = []
    for k in range(cm_n):
        comb_prob.append(prob_list[comb_indx[k]])
    comb_prob = np.array(comb_prob)
    comb_prob_label = pd.DataFrame(comb_prob.mean(axis=0).argmax(axis=1), columns=['Category'])
    ID = pd.DataFrame(np.arange(500), columns=['ID'])
    out_csv = pd.concat([ID, comb_prob_label], axis=1)
    print("Essemble model: "+index_name)
    out_csv.to_csv("team_6_submission.csv", index=False)

In [5]:
name_list = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'yolov8']
prob_list = prob_eff
prob_list.append(yoloV)
essemble_comb([2,3,6],name_list,prob_list)

Essemble model: B2B3yolov8
